In [1]:
import onnx
from onnx import TensorProto
from onnx.mapping import TENSOR_TYPE_TO_NP_TYPE


In [18]:
def type_conv(t): # see https://github.com/onnx/onnx/blob/master/onnx/mapping.py#L12
    dt = TENSOR_TYPE_TO_NP_TYPE[t]
    dt = dt.name.upper()
    dt = dt.replace("FLOAT", "FP")
    return f"TYPE_{dt}"

def node_dims(node, include_batch):
    shape = node.type.tensor_type.shape.dim
    dims = []
    for i, d in enumerate(shape):
        if not include_batch and i == 0: continue
        if d.HasField("dim_param"): dims += [-1]
        elif d.HasField("dim_value"): dims += [d.dim_value]
        else: raise(Exception(f"Unknown dimension {d}"))
    s = ", ".join(str(d) for d in dims)
    if not include_batch and not len(dims):
        return ("dims: [ 1 ]\n" + 
               "      reshape: { shape: [ ] }")
    s = f"dims: [ {s} ]"
    return s

def node_str(node, include_batch=False):
    return \
f"""
    {{
      name: "{node.name}"
      data_type: {type_conv(node.type.tensor_type.elem_type)}
      {node_dims(node, include_batch)}
    }}"""

def graph_str(graph, include_batch=False):
    s = ""
    s += "input ["
    t = [node_str(n, include_batch) for n in graph.input]
    s += ",\n".join(t)
    s += "\n]\n\n"

    s += "output ["
    t = [node_str(n, include_batch) for n in graph.output]
    s += ",\n".join(t)
    s += "\n]"

    return s

def header_str(model_onnx, max_batch=None):
    s = 'platform: "onnxruntime_onnx"\n'
    if max_batch is not None: s+= f'max_batch_size: {max_batch}\n'
    s += "\n"
    return s

def onnx_to_str(f, max_batch=None):
    model_onnx = onnx.load(f)
    graph = model_onnx.graph
    return header_str(model_onnx, max_batch) + graph_str(graph, include_batch=max_batch is None)

In [19]:
print(onnx_to_str("fastpitch.onnx", 16))

platform: "onnxruntime_onnx"
max_batch_size: 16

input [
    {
      name: "text"
      data_type: TYPE_INT64
      dims: [ -1 ]
    }
]

output [
    {
      name: "spect"
      data_type: TYPE_FP32
      dims: [ 80, -1 ]
    },

    {
      name: "num_frames"
      data_type: TYPE_INT64
      dims: [ 1 ]
      reshape: { shape: [ ] }
    },

    {
      name: "durs_predicted"
      data_type: TYPE_FP32
      dims: [ -1 ]
    },

    {
      name: "log_durs_predicted"
      data_type: TYPE_FP32
      dims: [ -1 ]
    },

    {
      name: "pitch_predicted"
      data_type: TYPE_FP32
      dims: [ -1 ]
    }
]


In [23]:
with open("model_repository/fastpitch/config.pbtxt", "w") as cfg:
    cfg.write(onnx_to_str("fastpitch.onnx", 128))

with open("model_repository/hifigan/config.pbtxt", "w") as cfg:
    cfg.write(onnx_to_str("hifigan.onnx", 128))

In [18]:
node.type.tensor_type.shape.dim[0]

dim_param: "text_dynamic_axes_1"

In [22]:
!cp onnx-to-triton.ipynb /ext_home/workspace_ru/code